#Cracking the .spc binary format

No longer needed as I found the Python spc [library](https://github.com/rohanisaac/spc)

In [1]:
import spc

ftir_1 = spc.File('eosfp/CLYDE_NTEST445.SPC')

Read raw data

HEADER
Generated x-values

SUBFILE 0 
----------
Using global subpoints
Using global subpoints
Using global subpoints 5807
Using global subpoints 5807
Using global exponent 253
Extracted integer y-data


OverflowError: Python int too large to convert to C long

Previous attempt (which didn't get very far - couldn't find almost anything in the binary file other than the number of values and the initial wavenumber (400. something or other)

In [2]:
cd eosfp/

/home/clyde/Dropbox/Project Stuff/Notebooks/eosfp


In [3]:
import struct
import numpy as np

In [4]:
with open('CLYDE_~1.PRN') as f:
    contents = f.readlines()
data = np.array([l.split() for l in contents[:-1]], dtype=np.float)

In [190]:
data[:,1][:20]

array([ -5.55000000e-04,   9.79400000e-04,   4.66830000e-03,
         3.46740000e-03,  -7.50000000e-05,  -7.92330000e-03,
        -8.65190000e-03,   1.42150000e-03,  -1.59580000e-03,
        -7.87130000e-03,  -2.15130000e-03,   8.50740000e-03,
         5.30330000e-03,  -2.90400000e-03,  -1.50480000e-03,
         5.62580000e-03,   4.37490000e-03,  -2.65860000e-03,
        -6.40500000e-04,   2.05950000e-03])

In [6]:
len(data)

5807

In [7]:
cm_1__nm = lambda wnm: 1./wnm * 1e-2

In [8]:
with open('CLYDE_NTEST445.SPC','rb') as bf:
    bin_data = bf.read()

In [59]:
bin_data[512+32:512+32+4]

'\x82\x06\xef\xf0'

In [134]:
len(bin_data[512:23774])/4

5815

In [133]:
int('5cde',base=16)

23774

In [136]:
struct.unpack('>f',bin_data[512+32:512+32+4])[0]

-9.913623873634808e-38

In [ ]:
52 65 6C 54 69 6D 65 00 00 00 00

In [56]:
chr(int('00',base=16))

'\x00'

In [143]:
struct.unpack('>f',bin_data[512+32+12:512+32+12+4])[0]

9.039539337158203

In [200]:
h = '41 10 A1 F4'
b = '01000001' + '00010000' + '10100001' + '11110100'

p = 23

si = b[0]
e = b[1:9]
s = b[9:32]

#b_l = '11110100' + '10100001' + '00010000' + '01000001' 

si_b = b[24]
e_b = b[25:] + b[16:17]
s_b  = b[17:24] + b[8:16] + b[0:8]

print(int(si,base=2),int(e,base=2),int(s,base=2))

(0, 130, 1090036)


In [239]:
'p' * 5

'ppppp'

In [270]:
def proc_byte_str(b_str):
    orig_l = len(b_str)
    b_str = b_str.split('0b')[1]
    assert(len(b_str) == orig_l -2)

    if len(b_str)<8:
        b_str = '0' * (8-len(b_str)) + b_str
    return b_str

def bytes_to_bin(data,ind):
    b_vals = [struct.unpack('B',data[ind+i:ind+i+1])[0] for i in range(4)]
    rb_strs = [bin(v) for v in b_vals]
    pb_strs = [proc_byte_str(s) for s in rb_strs]
    return ''.join(pb_strs)

def bin_to_float(b_str, base=2, offset=127):
    p = 23
    si = b_str[0]
    e = b_str[1:9]
    s = b_str[9:32]

    return -1 **int(si,base=2) * (1+int(s,base=2)*base**(-1*p)) * 2**(int(e,base=2)-offset)

def b_to_l(b_str):
    return b_str[24] + b_str[25:] + b_str[16:17] + b_str[17:24] + b_str[8:16] + b_str[0:8]

def convert_bin(data, ind, base=2, offset=127, endian='big'):
    p_bin = bytes_to_bin(data, ind)
    
    if endian == 'little':
        p_bin = b_to_l(p_bin)
    
    f = bin_to_float(p_bin,base=base,offset=offset)
    return f

init_ind = 512+32
convert_bin(bin_data, init_ind, base=2, offset=127, endian='little')

#f(x) = ax +b
#f(n1)-f(n2) / n2-n1 = a
#f(n1) - f(n2) / n2-n1 == f(n3) - f(n4) / n4-n3

-5.9179827847889546e+29

In [235]:
bin(83)

'0b1010011'

In [228]:
int(bin(struct.unpack('b',bin_data[512+32+12:512+32+12+1])[0]), base=2)

65

In [204]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=2)-127)

-9.039539337158203

In [205]:
-1 **int(si_b,base=2) * (1+int(s_b,base=2)*2**(-1*p)) * 2**(int(e_b,base=2)-127)

-1.020861160410558e+32

In [11]:
#big endian
'82 06 EF F0'

'82 06 EF F0'

In [209]:
b_rep = '10000010' + '00000110' + '11101111' + '11110000'

In [210]:
p = 23
si = b_rep[0]
e = b_rep[1:9]
s = b_rep[9:32]

int(si,base=2),int(e,base=2),int(s,base=2)

(1, 4, 454640)

In [215]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=2)-127)

-1.5861798197815693e-36

In [212]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=4)-253)

-4.773236268906365e-72

In [213]:
p = 23
si = b_rep[0]
e = b_rep[1:10]
s = b_rep[10:32]

int(si,base=2),int(e,base=2),int(s,base=2)

(1, 8, 454640)

In [91]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=2)-127)

-1.5861798197815693e-36

In [15]:
#-1**int(si,base=2)*sum([int(b)*2**(-i) for i,b in enumerate('1'+s)]) * 2**(int(e,base=2)-127)

In [16]:
struct.unpack('<f',bin_data[512+32:512+32+4])[0]

-5.9179827847889546e+29

In [ ]:
#little endian
'F0 EF 06 82'

In [166]:
b_rep_l = '11110000' + '11101111' + '00000110' + '10000010'

In [167]:
rev = lambda s: ''.join(reversed(s))

In [168]:
p = 23
si = b_rep_l[0]
e = b_rep_l[1:9]
s = b_rep_l[9:32]

int(si,base=2),int(e,base=2),int(s,base=2)

(1, 225, 7276162)

In [169]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=2)-127)

-5.9179827847889546e+29

In [122]:
-1 **int(si,base=8) * (1+int(s,base=8)*8**(-1*p)) * 8**(int(e,base=2)-253)

-6.586733212393975e+178

In [119]:
p = 23
si = b_rep_l[0]
e = b_rep_l[1:10]
s = b_rep_l[10:32]

int(si,base=2),int(e,base=2),int(s,base=2)

(1, 451, 3081730)

In [120]:
-1 **int(si,base=2) * (1+int(s,base=2)*2**(-1*p)) * 2**(int(e,base=2)-462)

-0.0006676615448668599

In [33]:
locs = []

#possible formatsdata.shape[0]
fmts = 'dqQfiIlLhH'
#their corresponding sizes in bytes
szs = [8,8,8,4,4,4,8,8,2,2]

search_values = [data.shape[0],
                 cm_1__nm(data[0][0]),
                 data[0][1],
                 cm_1__nm(data[1][0]),
                 data[1][1],
                 cm_1__nm(data[2][0]),
                 data[2][1],
                 data[1][0]-data[0][0],
                 401,
                 402,
                 403,
                 404,
                 405,
                 406]


for search_value in search_values:
    for i in xrange(len(bin_data)-max(szs)):
        quants = [struct.unpack(fmt,bin_data[i:i+sz])[0] for fmt,sz in zip(fmts,szs)]
        quants = np.array(quants)

        try:
            q_loc = np.where(abs(quants - search_value) < float(search_value)/1000)[0][0]
            locs.append((search_value,i,fmts[q_loc]))
        except IndexError:
            pass

/home/clyde/anaconda/envs/dev/lib/python2.7/site-packages/IPython/kernel/__main__.py:30: RuntimeWarning: invalid value encountered in less
/home/clyde/anaconda/envs/dev/lib/python2.7/site-packages/IPython/kernel/__main__.py:30: RuntimeWarning: invalid value encountered in subtract


In [34]:
locs

[(5807, 4, 'i'),
 (2.486287026864831e-05, 22467, 'f'),
 (401, 1982, 'h'),
 (401, 3090, 'h'),
 (401, 5450, 'h'),
 (402, 3086, 'h'),
 (402, 3222, 'h'),
 (402, 3290, 'h'),
 (402, 4190, 'h'),
 (402, 4478, 'h'),
 (402, 15270, 'h'),
 (403, 3166, 'h'),
 (403, 3282, 'h'),
 (403, 3654, 'h'),
 (403, 4118, 'h'),
 (403, 8238, 'h'),
 (403, 13246, 'h'),
 (403, 13250, 'h'),
 (403, 13638, 'h'),
 (404, 3250, 'h'),
 (404, 3270, 'h'),
 (404, 3330, 'h'),
 (404, 3438, 'h'),
 (404, 3874, 'h'),
 (404, 5570, 'h'),
 (404, 12749, 'h'),
 (404, 13482, 'h'),
 (405, 2878, 'h'),
 (405, 3026, 'h'),
 (405, 3070, 'h'),
 (405, 3118, 'h'),
 (405, 4486, 'h'),
 (405, 4726, 'h'),
 (405, 4746, 'h'),
 (405, 12466, 'h'),
 (405, 12737, 'h'),
 (405, 13262, 'h'),
 (406, 3610, 'h'),
 (406, 3678, 'h'),
 (406, 4374, 'h'),
 (406, 5766, 'h'),
 (406, 7050, 'h'),
 (406, 19934, 'f')]

In [35]:
zip(zip(*locs)[0][19:],np.array(zip(*locs)[1])[19:]-5450)

[(404, -2200),
 (404, -2180),
 (404, -2120),
 (404, -2012),
 (404, -1576),
 (404, 120),
 (404, 7299),
 (404, 8032),
 (405, -2572),
 (405, -2424),
 (405, -2380),
 (405, -2332),
 (405, -964),
 (405, -724),
 (405, -704),
 (405, 7016),
 (405, 7287),
 (405, 7812),
 (406, -1840),
 (406, -1772),
 (406, -1076),
 (406, 316),
 (406, 1600),
 (406, 14484)]

switching to trying to crack the JCAMP data format.